In [ ]:
#最小距离分类器
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from numpy.ma.extras import vstack
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score

matplotlib.use("TkAgg")

data=rio.open('Flasshrc_zj.dat')
b1=data.read(1)
b2=data.read(2)
b3=data.read(3)
b4=data.read(4)
b5=data.read(5)
b6=data.read(6)
b7=data.read(7)

train_data=rio.open('train_ROI.dat')
train_roi=train_data.read(1)

In [2]:

#调出水体部分
train_water_mask=train_roi==1
b1_water=b1[train_roi==1]
b2_water=b2[train_roi==1]
b3_water=b3[train_roi==1]
b4_water=b4[train_roi==1]
b5_water=b5[train_roi==1]
b6_water=b6[train_roi==1]
b7_water=b7[train_roi==1]

train_water=np.vstack((b1_water,b2_water,b3_water,b4_water,b5_water,b6_water,b7_water))
train_water_center=train_water.mean(axis=1) #计算水体像元每个波段的平均值

#调出裸地部分
train_land_mask=train_roi==2
b1_land=b1[train_roi==2]
b2_land=b2[train_roi==2]
b3_land=b3[train_roi==2]
b4_land=b4[train_roi==2]
b5_land=b5[train_roi==2]
b6_land=b6[train_roi==2]
b7_land=b7[train_roi==2]
train_land=np.vstack((b1_land,b2_land,b3_land,b4_land,b5_land,b6_land,b7_land))
train_land_center=train_land.mean(axis=1) #计算水体像元每个波段的平均值

#调出植被部分
train_plant_mask=train_roi==3
b1_plant=b1[train_roi==3]
b2_plant=b2[train_roi==3]
b3_plant=b3[train_roi==3]
b4_plant=b4[train_roi==3]
b5_plant=b5[train_roi==3]
b6_plant=b6[train_roi==3]
b7_plant=b7[train_roi==3]
train_plant=np.vstack((b1_plant,b2_plant,b3_plant,b4_plant,b5_plant,b6_plant,b7_plant))
train_plant_center=train_plant.mean(axis=1) #计算水体像元每个波段的平均值

#调出建筑部分
train_building_mask=train_roi==4
b1_building=b1[train_roi==4]
b2_building=b2[train_roi==4]
b3_building=b3[train_roi==4]
b4_building=b4[train_roi==4]
b5_building=b5[train_roi==4]
b6_building=b6[train_roi==4]
b7_building=b7[train_roi==4]
train_building=np.vstack((b1_building,b2_building,b3_building,b4_building,b5_building,b6_building,b7_building))
train_building_center=train_building.mean(axis=1) #计算水体像元每个波段的平均值



In [ ]:
# 方法一 中间值法
image_class=np.zeros(b1.shape)
for row in range(b1.shape[0]):  #逐行取出像元
    for col in range(b1.shape[1]): #逐列取出像元
        r=np.array([b1[row,col],b2[row,col],b3[row,col],b4[row,col],
                    b5[row,col],b6[row,col],b7[row,col]])
        dist_water=np.sum((r-train_water_center)**2) #计算像元到水体中间的距离
        dist_plant = np.sum((r - train_plant_center) ** 2)
        dist_land = np.sum((r - train_land_center) ** 2)
        dist_building = np.sum((r - train_building_center) ** 2)

        distance=np.array([dist_land,dist_building,dist_plant,dist_water])
        min_distance=distance.argmin()

        image_class[row,col]=min_distance+1

    print(row/b1.shape[0]) #展示图像分类的进度
plt.figure()
plt.imshow(image_class)
plt.show()


0.0
0.00044943820224719103
0.0008988764044943821
0.001348314606741573
0.0017977528089887641
0.0022471910112359553
0.002696629213483146
0.003146067415730337
0.0035955056179775282
0.004044943820224719
0.0044943820224719105
0.004943820224719101
0.005393258426966292
0.005842696629213483
0.006292134831460674
0.006741573033707865
0.0071910112359550565
0.007640449438202248
0.008089887640449439
0.00853932584269663
0.008988764044943821
0.00943820224719101
0.009887640449438202
0.010337078651685393
0.010786516853932584
0.011235955056179775
0.011685393258426966
0.012134831460674157
0.012584269662921348
0.01303370786516854
0.01348314606741573
0.013932584269662922
0.014382022471910113
0.014831460674157304
0.015280898876404495
0.015730337078651686
0.016179775280898877
0.01662921348314607
0.01707865168539326
0.01752808988764045
0.017977528089887642
0.018426966292134833
0.01887640449438202
0.019325842696629212
0.019775280898876403
0.020224719101123594
0.020674157303370785
0.021123595505617977
0.0215730

In [ ]:
# 计算中间值
plt.figure()
for i in range(17641):
    plt.plot(train_water[:,i],c='blue')
plt.plot(train_water_center,c='red')
plt.show()

In [ ]:
X=np.hstack((train_water,train_plant,train_building,train_land)).T

y_water=np.ones(train_water.shape[1])
y_plant=np.ones(train_plant.shape[1])*2
y_building=np.ones(train_building.shape[1])*3
y_land=np.ones(train_land.shape[1])*4
y=np.hstack((y_water,y_plant,y_building,y_land))


In [8]:
# 方法二：随机森林
model=RandomForestClassifier().fit(X,y) #当学习完毕后，就可以将其保存为一个model
test_x=np.vstack((b1.flatten(),b2.flatten(),b3.flatten(),b4.flatten(),
                  b5.flatten(),b6.flatten(),b7.flatten())).T
y_pred=model.predict(test_x)
image_class=y_pred.reshape(b1.shape)
plt.figure()
plt.imshow(image_class)
plt.show()

In [7]:
model=GradientBoostingClassifier().fit(X,y) #当学习完毕后，就可以将其保存为一个model
test_x=np.vstack((b1.flatten(),b2.flatten(),b3.flatten(),b4.flatten(),
                  b5.flatten(),b6.flatten(),b7.flatten())).T
y_pred=model.predict(test_x)
image_class=y_pred.reshape(b1.shape)
plt.figure()
plt.imshow(image_class)
plt.show()


In [ ]:
# 仅在训练数据上评估模型性能
y_train_pred = model.predict(X)
print('训练数据分类准确率为：', accuracy_score(y, y_train_pred))
print('训练数据宏平均准确率：', precision_score(y, y_train_pred, average='macro'))
print('训练数据微平均准确率：', precision_score(y, y_train_pred, average='micro'))